In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import subprocess
import time
from dataclasses import dataclass
import ipywidgets
from ipywidgets import interact,widgets
from ipywidgets import Button, Layout
from ovito.io import import_file
from ovito.modifiers import PolyhedralTemplateMatchingModifier
from ovito.vis import *
from ovito.pipeline import *

In [3]:
### Function
## Common
def copy_file(source, target):
    """
    Copy a file from the source path to the target path
    """
    copy = f'cp {source} {target}'
    run = subprocess.Popen(copy, shell=True)
    run.wait()
        
def Checkdir(keyword):
    '''
    Check whether the file with corresponding key already existed.
    When the file existed, it will be removed
    '''
    root = os.getcwd()
    dir_list = os.listdir("./")
    for path in dir_list:
        if keyword in path:
            dir_path = f"{root}/{path}"
            if os.path.exists(dir_path):
                print(f"The dictionary {dir_path} will be removed!")
                shutil.rmtree(dir_path)
    
    
def showtime(func):
    def wrapper(System):
        start_time = time.time()
        func(System)
        end_time = time.time()
        print('Running time is {} s'.format(end_time - start_time))
    return wrapper


def check_value_type(value, value_type):
    if isinstance(value, value_type):
        pass
    else:
        print(f"The {value} has wrong data type, It must be {value_type}.")
    

def change_symbol(value):
    if "," in str(value):
        value_new = str(value).strip().replace(",", ".")
        return float(value_new)
    else:
        return float(str(value).strip())


## Lammps simulation
def copy_potential(System):
    copy_file(f"./potentials/{System.potential_name}", f"./{System.Project_name}/{System.potential_name}")

def write_input(System):
    """
    Problem: NPT or NVT
    """
    with open(f"./{System.Project_name}/lammps_input", "w+") as fw:
        fw.write(f"""
dimension 3
units metal
atom_style atomic
boundary p p p

lattice fcc {System.lattice_constant}
region box block 0 {System.box_length} 0 {System.box_length} 0 {System.box_length}
create_box 1 box
create_atoms 1 box
mass 1 {System.lattice[2]}

pair_style eam/alloy
pair_coeff * * {System.potential_name} {System.element}

neighbor 3.0 bin

minimize 1e-8 1e-8 10000 100000
min_style cg

write_data original

change_box all x delta 0 3 
change_box all y delta 0 3 
change_box all z delta 0 3 boundary s s s

timestep {System.timestep}

thermo {System.thermo_time}

thermo_style custom step temp vol press

velocity all create {System.start_temperature} {System.random_number} mom yes rot yes dist gaussian

dump 1 all custom 1000 melting id type xs ys zs

fix 2 all nvt temp {System.start_temperature} {System.end_temperature} 0.1

run {System.running_steps}
        """)


@showtime
def run_lammps(input_file):
    serial_run = 'mpirun -n 2 lmp_mpi <{} > log'.format(str(input_file))
    run = subprocess.Popen(serial_run, shell=True)
    run.wait()


#@showtime
def animate_show(particle_size=0.5):
    traj = read(filename=f"./melting", format="lammps-dump-text", index=":")
    write(f"./melting.traj", traj)
    import nglview
    traj = Trajectory("./melting.traj")
    movie = nglview.show_asetraj(traj)
    movie.display(gui=True, style='ngl')
    movie.add_cartoon(selection='all')
    movie.add_spacefill(radius_type="vdw", scale=0.5, radius=0.5, color='red', max_frame=500)
    movie.remove_ball_and_stick()
    movie.add_unitcell()
    movie.camera = "orthographic"
    return movie

In [4]:
# Dataclass
@dataclass
class MD_system:
    Project_name: str ="Melting"
    element:str ="Al"
    lattice_constant: float = None
    box_length: int = 3
    timestep: float = 0.001
    start_temperature: int = None
    seed: int = 100
    end_temperature: int =None
    thermo_time :int = 5000
    running_steps :int = 150000
    
    @property
    def potential_name(self):
        if self.element == "Al":
            potential_name = "Al03.eam.alloy"
        if self.element == "Cu":
            potential_name = "Cu01.eam.alloy"
        return potential_name
        
    @property
    def lattice(self):
        def atom_info(self):
            Al = {"a":[3.90, 4.10], "mass":26.982, "melting_point":933.44}
            Cu = {"a":[3.62, 3.72], "mass":63.546, "melting_point":1360}
            atoms = vars()[self.element]
            return atoms

        self.atoms = atom_info(self)
        atom_mass = self.atoms["mass"]
        alat_min, alat_max = self.atoms['a'][0], self.atoms['a'][1]
        melting_point = self.atoms["melting_point"]    
        return alat_min, alat_max, atom_mass, melting_point
    
    @property
    def random_number(self):
        seed = np.int64(self.seed)
        np.random.seed(seed)
        return int("".join([str(x) for x in np.random.randint(10, size=6)]))

In [5]:
# initialize the MD_System
System_melting = MD_system()

title_show = widgets.HTML(value="</h1>The Molecule Dynamic Simulation of Melting</h1>", layout=Layout(width='50%', height='50px', fontsize=50))
title_show.style.text_align='center'

# define element list
elements_show = widgets.ToggleButtons(
    options=['Al', 'Cu'],
    value = 'Al',
    description='Elements:',
    disabled=False,
    button_style='success',
    layout=Layout(width='50%', height='50px', fontsize=50)
)

aLat_show = widgets.SelectionSlider(
    options=[None],
    description = 'Lattice constant',
    disabled = False,
    orientation = 'horizontal',
    layout=Layout(width='30%', height='50px')
)

melting_point_show = widgets.HTML(layout=Layout(width='50%', height='50px', fontsize=50))

# Change the lattice constant and atom mass corresponding with the name of element
def on_element_change(change):
    System_melting.element=elements_show.value
    with aLat_show.hold_trait_notifications():
        aLat_show.options = np.linspace(System_melting.lattice[0], System_melting.lattice[1], num=20)
    melting_point_show.value = f"The melting point of element {System_melting.element} is {System_melting.lattice[3]} K"

elements_show.observe(on_element_change, "value")

# define Project name
Project_name_show = widgets.Text(
    value='Your Project name',
    placeholder='Type something',
    description='Project name:',
    layout=Layout(width='50%', height='50px'),
    disabled=False
)


# define box_length 
box_length_show = widgets.BoundedIntText(
    value=4,
    min=3,
    max=6,
    step=1,
    description='Box length:',
    layout=Layout(width='30%', height='50px'),
    disabled=False
)

# set up the button for submission
submit_buttom = widgets.Button(
    description='submit',
    layout=Layout(width='30%', height='50px'),
    button_style='success'
)



start_T_show = widgets.Text(
    value='Temperature',
    placeholder='Type something',
    description='Starting temprature (K) is:',
    layout=Layout(width='30%', height='50px'),
    disabled=False
)

end_T_show = widgets.Text(
    value='Temperature',
    placeholder='Type something',
    description='End temprature (K) is:',
    layout=Layout(width='30%', height='50px'),
    disabled=False
)

seed_show = widgets.Text(
    value='Seed',
    placeholder='Type something',
    description='Seed is:',
    layout=Layout(width='30%', height='50px'),
    disabled=False
)

# define a function which can obtain the value form the interactive surface
def buttom_click(sender):
    System_melting.Project_name = Project_name_show.value
    System_melting.box_length = box_length_show.value
    System_melting.lattice_constant = aLat_show.value
    System_melting.start_temperature = start_T_show.value
    System_melting.end_temperature = end_T_show.value
    System_melting.seed = seed_show.value
    
# connect the function with the button
submit_buttom.on_click(buttom_click)

# show the widgets
box = widgets.VBox([title_show, Project_name_show, elements_show, box_length_show, aLat_show, melting_point_show, start_T_show, end_T_show, seed_show, submit_buttom])
box.layout=Layout(width='1080px', height='720px')
display(box)# aLat_show

In [7]:
# Prepare Input file
os.mkdir(f"./{System_melting.Project_name}")
copy_potential(System_melting)
write_input(System_melting)

# Run Lammps
os.chdir(f"./{System_melting.Project_name}")
run_lammps("lammps_input")

# Show the animation
### Here: pls write a function for playing!!!
pipeline = import_file("./melting", multiple_frames=True)
pipeline.add_to_scene()
vp = Viewport(type=Viewport.Type.Ortho, camera_dir=(2, 2, -1))
vp.zoom_all()

max_frame = pipeline.source.num_frames
play_image = widgets.Play(
    value=10,
    min=300,
    max=599,
    step=1,
    description="Press play",
    disabled=False
)

slider = widgets.FloatProgress(
    value=10,
    min=300,
    max=599.0,
    step=1,
    description='Loading:',
    bar_style='success',
    orientation='horizontal'
)

widgets.jslink((play_image, 'value'), (slider, 'value'))


def play(vp, x):
    vp.dataset.anim.current_frame=x
    w.refresh()
    
w = vp.create_jupyter_widget()
w.layout = ipywidgets.Layout(width='1080px', height='720px')

widgets.interactive(play, x=play_image, vp=fixed(vp))
a = widgets.HBox([play_image, slider])
display(w, a)

'Cu01.eam.alloy'